# Audio Frontend for Xylo-A3 with AGC

In the other notebook (in this folder), we investigated the simulation of modules in the frontend of Xylo-A3, consisting of the PDM microphone.
This corresponds to the first signal path from the analog audio signal until the input of filterbank. We called this module PDM ADC.

In Xylo-A3, we have another signal path which starts from an ordinary microphone and consists of following modules:
- ordinary microphone producing an analog signal
- analog frontend consisting of fixed amplifier + anti-aliasing 1st order low-pass filter.
- AGC (automatic gain controller) module consisting of
  - PGA (programmable gain amplifier) whose gain can be adjusted in the range [1,30] with a 4-bit index control signal, which yields 16 possible gain levels.
  - Equivalent ADC implamented in two steps to eliminate the aliasing noise as much as possible:
    - high-rate ADC: to reduce anti-aliasing noise, the analog signal is sampled with 1, 2, or 4 times target audio sampling rate.
    - anti-aliasing + decimation filter: the resulting high-rate signal is then low-pass filtered and decimated to obtain a 10-bit signal.
  - AGC: output the equ-ADC is forwarded to an AGC module which measures the signal level and if it is low it sends control commands to PGA to increase the amplification gain. Also AGC makes sure that the signal amplification is reduces as soon as signal level goes up to avoid truncating the signal.
  
One can imagine the whole signal path from the output of the microphone until the output of the AGC and an AGC-ADC which quantizes the signal while adjusting its amplitude adaptively.

The output of this module is a 10-bit signal which is zero-padded by 4-bits to yield a 14-bit signal (the same as PDM-ADC), which is then forwarded to the filterbank module.

We already investigated the remaining modules (filterbank + divisive normalization + spike generation) in the other notebook. In this notebook, we will focus on the AGC-ADC.

## AGC Simulation
To see the effect of AGC, we will build a fake audio time-series and will modify its gain to mimic the effect of audio source coming close to or far from the microphone.
Then we will investigate how this signal is processed by the AGC frontend.

In [1]:
from rockpool.devices.xylo.xylo_a3.xylo_a3_sim.agc.adc import ADC
from rockpool.devices.xylo.xylo_a3.xylo_a3_sim.agc.amplifier import Amplifier
from rockpool.devices.xylo.xylo_a3.xylo_a3_sim.agc.envelope_controller import EnvelopeController
from rockpool.devices.xylo.xylo_a3.xylo_a3_sim.agc.gain_smoother import GainSmootherFPGA
from rockpool.devices.xylo.xylo_a3.xylo_a3_sim.agc.agc_frontend import AGC_ADC

from rockpool.devices.xylo.xylo_a3.xylo_a3_sim.agc.xylo_a3_agc_specs import AUDIO_SAMPLING_RATE, XYLO_MAX_AMP, HIGH_PASS_CORNER, LOW_PASS_CORNER

from rockpool.timeseries import TSContinuous
import numpy as np

import matplotlib.pyplot as plt

from scipy.io import wavfile
import os



def test_agc_audio():
    """
    this module tests the basic model of AGC to see if it works well.
    """
    # ================================================
    # *       build distance time-series
    # ================================================
    distance_fs = 6*AUDIO_SAMPLING_RATE
    duration = 1
    distance_time = np.arange(0, duration, step=1 / distance_fs)

    # at ref-distance from microphone the received signal will be rail-to-rail
    ref_distance = 1.0

    # * Model 2: abrupt movement in front of microphone
    distance = ref_distance * np.ones_like(distance_time)
    distance[duration/3 < distance_time < 2*duration/3] *= 4

    distance_ts = TSContinuous.from_clocked(distance, dt=1 / distance_fs, periodic=True)

    # ================================================
    # *     build an audio signal
    # ================================================
    freq = 1000
    
    # original audio signal
    audio = XYLO_MAX_AMP * np.sin(2*np.pi*freq*distance_time)
    
    # audio affected by variation of the distance to the microphone
    power_attenuation_factor = 4
    amplitude_attenuation_factor = power_attenuation_factor/2
    audio_modified = audio * (ref_distance / distance**amplitude_attenuation_factor)
    

    # ================================================
    # *     build an ADC
    # ================================================
    num_bits = 10
    adc_oversampling_factor = 2     # options: 1, 2, 4 in the current HW design
    adc = ADC(
        num_bits=num_bits,
        max_audio_amplitude=XYLO_MAX_AMP,
        oversampling_factor=adc_oversampling_factor,
        fs=AUDIO_SAMPLING_RATE,
    )

    # =======================================================
    # *   build an envelope estimator 
    # NOTE: we need to specify two time constants for this depending on how fast we would like the envelope controller
    # to respond to signal envelope variations.
    #
    # We strongly encourage reading the design manual of envelope controller in
    # https://spinystellate.office.synsense.ai/saeid.haghighatshoar/agc-for-xylo-v3
    #
    # `Rule of Thumb`:
    # An important parameter is fall_time_constant which specifies if siganl amplitude goes down how fast the reaction time of AGC
    # needs to be to adjust/amplify the signal amplitude. 
    #
    # This typically should be of the order of several seconds so that AGC does not confuse the silent slices of the audio signal
    # as attenuated audio.
    #
    # =======================================================
    rise_time_constant = 0.1e-3
    fall_time_constant = 1000e-3
    num_bits_command = 4

    max_PGA_gain = 32

    gain_ratio = max_PGA_gain ** (1 / 2**num_bits_command)
    saturation_level = int(min([1 / gain_ratio, 0.5]) * 2 ** (num_bits - 1))

    # we use a square-root waiting time for AGC
    # As a first use, one may use envelope controller with default parameters 
    waiting_time_vec = fall_time_constant * np.sqrt(
        np.arange(1, 2**num_bits_command + 1)
    )

    # envelope controller
    envelope_controller = EnvelopeController(
        num_bits=num_bits,
        saturation_level=saturation_level,
        max_PGA_gain=max_PGA_gain,
        rise_time_constant=rise_time_constant,
        fall_time_constant=fall_time_constant,
        reliable_max_hysteresis=5,
        num_bits_command=num_bits_command,
        waiting_time_vec=waiting_time_vec,
        fs=AUDIO_SAMPLING_RATE,
    )

    # =======================================================
    # *                  build an amplifier
    # =======================================================
    high_pass_corner = HIGH_PASS_CORNER     # around 50 Hz
    low_pass_corner = LOW_PASS_CORNER       # around 20 KHz
    fixed_gain_for_PGA_mode = False  # PGA is in AGC mode
    amplifier = Amplifier(
        high_pass_corner=high_pass_corner,
        low_pass_corner=low_pass_corner,
        pga_gain_vec=envelope_controller.pga_gain_vec,
        settling_time=0.0,
        fixed_gain_for_PGA_mode=fixed_gain_for_PGA_mode,
        fs=distance_fs,     # here 6 times the target audio sampling rate
    )

    # =======================================================
    # *          build a gain smoother module
    # =======================================================
    num_bits_gain_quantization = 10
    gain_smoother = GainSmootherFPGA(
        num_bits=num_bits,
        min_waiting_time=envelope_controller.waiting_time_vec.min(),
        num_bits_command=num_bits_command,
        pga_gain_vec=amplifier.pga_gain_vec,
        num_bits_gain_quantization=num_bits_gain_quantization,
        fs=AUDIO_SAMPLING_RATE,
    )

    # =======================================================
    # *       build an ADC with AGC capabilities
    # =======================================================
    adc_agc = AGC_ADC(
        amplifier=amplifier,
        adc=adc,
        envelope_controller=envelope_controller,
        gain_smoother=gain_smoother,
    )

    # =======================================================
    # *            how to run the simulations
    # =======================================================
    record = True
    progress_report = True

    # simulate the system
    adc_agc.reset()

    # continue
    agc_out, state = adc_agc.evolve(
        audio=audio_modified,
        audio_sample_rate=distance_fs,
        record=record,
        progress_report=progress_report,
    )

    # ===========================================================================
    #                         Plot and save the results
    # ===========================================================================
    plt.figure(figsize=(10, 16))

    plt.subplot(511)
    plt.plot(distance_time, distance)
    plt.ylabel("distance from microphone (m)")
    plt.grid(True)
    plt.title(
        f"AGC: bw:{[int(amplifier.high_pass_corner), int(amplifier.low_pass_corner)]} Hz, max amplitude:{adc.max_audio_amplitude},\nnum-bits:{adc.num_bits}"
    )

    plt.subplot(512)
    audio_norm = XYLO_MAX_AMP * audio / np.max(np.abs(audio))
    plt.plot(np.linspace(0, duration, len(audio_norm)), audio_norm)
    plt.plot([0, duration], [XYLO_MAX_AMP, XYLO_MAX_AMP], "k", linewidth=3)
    plt.plot([0, duration], [-XYLO_MAX_AMP, -XYLO_MAX_AMP], "k", linewidth=3)
    plt.grid(True)
    plt.ylabel(f"audio at distance {ref_distance} m")

    plt.subplot(513)
    plt.plot(np.linspace(0, duration, len(audio_modified)), audio_modified)
    plt.plot([0, duration], [XYLO_MAX_AMP, XYLO_MAX_AMP], "k", linewidth=3)
    plt.plot([0, duration], [-XYLO_MAX_AMP, -XYLO_MAX_AMP], "k", linewidth=3)
    plt.grid(True)
    plt.ylabel("audio affected by distance")

    plt.subplot(514)
    max_adc_val = 2 ** (adc.num_bits - 1) - 1
    min_adc_val = -max_adc_val

    plt.plot(np.linspace(0, duration, len(agc_out)), agc_out, label="adc-out")
    plt.plot(
        np.linspace(0, duration, len(state["gain_smoother_output"])),
        state["gain_smoother_output"],
        label="smoothed",
    )
    plt.plot(
        np.linspace(0, duration, len(state["envelope"])),
        state["envelope"],
        label="envelope",
    )
    plt.plot([0, duration], [max_adc_val, max_adc_val], "k", linewidth=3)
    plt.plot([0, duration], [min_adc_val, min_adc_val], "k", linewidth=3)
    plt.legend()
    plt.grid(True)
    plt.ylabel("output of ADC")

    plt.subplot(515)
    plt.plot(
        np.linspace(0, duration, len(state["agc_pga_gain"])), state["agc_pga_gain"]
    )
    plt.grid(True)
    plt.ylabel("PGA gain")

    plt.xlabel("time (sec)")
    plt.draw()
    
    plt.show()


def main():
    test_agc_audio()
    # test_dataset()


if __name__ == "__main__":
    main()

RuntimeError: This version of jaxlib was built using AVX instructions, which your CPU and/or operating system do not support. You may be able work around this issue by building jaxlib from source.

--- 